In [4]:
import pickle
import json
from nuscenes.nuscenes import NuScenes

In [12]:
nusc = NuScenes(version='v1.0-test', dataroot='../data/nuscenes-test/', verbose=True)

Loading NuScenes tables for version v1.0-test...
23 category,
8 attribute,
4 visibility,
0 instance,
12 sensor,
1800 calibrated_sensor,
462901 ego_pose,
15 log,
150 scene,
6008 sample,
462901 sample_data,
0 sample_annotation,
4 map,
Done loading in 5.355 seconds.
Reverse indexing ...
Done reverse indexing in 0.9 seconds.


In [58]:
with open('../data/nuscenes/nuscenes2d_temporal_infos_test.pkl', 'rb') as f:
	val = pickle.load(f)

In [59]:
with open('../test/stream_petr_vov_flash_800_bs2_seq_24e/Sun_Nov_12_15_32_19_2023/pts_bbox/results_nusc.json', 'rb') as f:
	res = json.load(f)

In [13]:
nbr_scence = len(nusc.scene)
nbr_scence

150

In [14]:
def scence_plus_samples(nbr_scence):
    new_scence_json = {}
    for i in range(nbr_scence):
        
        my_scene = nusc.scene[i]
        first_sample_token = my_scene['first_sample_token']
        last_sample_token = my_scene['last_sample_token']
        my_scene['list_samples'] = [first_sample_token]
        new_scence_json[my_scene['token']] = my_scene
        sample_token = first_sample_token
        
        while True:
            my_sample = nusc.get('sample', sample_token)
            sample_token = my_sample['next']
            my_scene['list_samples'].append(sample_token)
            if sample_token == last_sample_token:
                break
    return new_scence_json

In [15]:
new_scence_json = scence_plus_samples(nbr_scence)

In [16]:
with open('scence+smaples_test.json', 'w') as f:
    json.dump(new_scence_json, f, indent=4)

In [17]:
def link_frame_to_scence(detection_result):
    
    detection_result = iter(detection_result)
    present_sample_key = next(detection_result)
    scence_plus_frames = {}
    
    while True:
        present_sample = nusc.get('sample', present_sample_key)
        present_scence_key = present_sample['scene_token']
        if present_scence_key not in scence_plus_frames:
            scence_plus_frames[present_scence_key] = []
            
        scence_plus_frames[present_scence_key].append(present_sample_key)
        try:
            present_sample_key = next(detection_result)
        except StopIteration:
            print("Mapping Finish without Error.")
            break
    
    return scence_plus_frames

In [18]:
scence_plus_frames = link_frame_to_scence(res['results'])

NameError: name 'res' is not defined

In [90]:
len(scence_plus_frames['c3ab8ee2c1a54068a72d7eb4cf22e43d'])

39

In [89]:
with open('scence+frames.json', 'w') as f:
    json.dump(scence_plus_frames, f, indent=4)

In [87]:
next(iter(res['results']))

'30e55a3ec6184d8cb1944b39ba19d622'